In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
client_id = 'CLIENT_ID'
client_secret = 'CLIENT_SECRET'
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
import os
import random
from midi2audio import FluidSynth
from pydub import AudioSegment
import json

In [ ]:
genres = os.listdir("./MIDIs")
instruments = os.listdir("./instruments")

In [ ]:
key_dict = {
    0: 'C', 1: 'C#',
    2: 'D', 3: 'D#',
    4: 'E',
    5: 'F', 6: 'F#',
    7: 'G', 8: 'G#',
    9: 'A', 10: 'A#',
    11: 'B'
}

mode_dict = {
    0: 'Minor', 1: 'Major'
}

In [ ]:
os.makedirs('tmp', exist_ok=True)
os.makedirs('renders', exist_ok=True)
id = 0
for _ in range(1):
    # Pick song
    general_genre = random.choice(genres)
    specific_genre = random.choice(os.listdir(f"./MIDIs/{general_genre}"))
    artist = random.choice(os.listdir(f"./MIDIs/{general_genre}/{specific_genre}"))
    song = random.choice(os.listdir(f"./MIDIs/{general_genre}/{specific_genre}/{artist}"))

    # Render MIDI
    instrument = random.choice(instruments)
    fs = FluidSynth(f'./instruments/{instrument}', sample_rate=44100)
    fs.midi_to_audio(f'./MIDIs/{general_genre}/{specific_genre}/{artist}/{song}',
                     './tmp/render.wav')

    # Find it on Spotify
    spotify_track = sp.search(artist + " " + song[:-4], limit=1, type='track')
    if len(spotify_track['tracks']['items']) == 0: continue
    track_id = spotify_track['tracks']['items'][0]['id']

    # Get Spotify's metadata
    spotify_analysis = sp.audio_analysis(track_id)
    for section in spotify_analysis["sections"]:
        data = {
            'instrument': instrument[:-4],
            'genre': specific_genre,
            'tempo': int(section['tempo']),
            'key': key_dict[section['key']],
            'mode': mode_dict[section['mode']],
            'loudness': section['loudness']
        }

        # Cut section of rendered audio
        audio = AudioSegment.from_wav('./tmp/render.wav')
        start = section['start']*1000
        end = section['start']*1000 + section['duration']*1000
        cut_audio = audio[start:end]

        # TODO: Add filters (wp eps)
        
        # Save section and metadata
        cut_audio.export(f'./renders/{id}.wav', format='wav')
        with open(f'./renders/{id}.json', 'w') as json_file:
            json.dump(data, json_file)
        id += 1
    os.remove(f'./tmp/render.wav')
os.rmdir('tmp')